In [13]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import papermill as pm

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus  import stopwords
import re

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import re

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prabinapokharel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/prabinapokharel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prabinapokharel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
N = 3

In [30]:
def generate_ngram_features(data, n):
    vectorizer = CountVectorizer(ngram_range=(n, n), binary=True)
    X = vectorizer.fit_transform(data)
    feature_names = vectorizer.get_feature_names_out()
    return X, feature_names

In [31]:
df = pd.read_csv("data/submission_metadata.csv")
df.head()

,Name,Student ID,Email,Status,Submission ID,Total Score,Max Points,Question 1 Score,Question 1 Weight,Question 1 Graded?,Question 1 Response,Question 1 Submitted At
0,Andrew Pan,A10001,andrep24@uw.edu,Ungraded,231798102,0.0,1.0,NaN,1.0,False,Binary search trees represent a sophisticated ...,2024-02-17 14:04:35 -0800


In [32]:
text_data = df['Question 1 Response'].tolist()

['Binary search trees represent a sophisticated data structure pivotal for managing and manipulating large datasets with optimal efficiency. Their function transcends simple storage; they offer a systematic arrangement of elements, allowing logarithmic time complexity for search operations through efficient partitioning of the data space. Furthermore, binary search trees facilitate dynamic operations such as insertion and deletion while maintaining their balanced structure, making them indispensable in domains where performance and scalability are paramount, such as database management systems and network routing algorithms.']

In [33]:
X, feature_names = generate_ngram_features(text_data, N)
ngram_df = pd.DataFrame(X.toarray(), columns=feature_names)
ngram_df

,allowing logarithmic time,and deletion while,and manipulating large,and network routing,and scalability are,are paramount such,arrangement of elements,as database management,as insertion and,balanced structure making,...,them indispensable in,they offer systematic,through efficient partitioning,time complexity for,transcends simple storage,trees facilitate dynamic,trees represent sophisticated,where performance and,while maintaining their,with optimal efficiency
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [21]:
def generate_ngram_tfidf_pipeline(data, labels, n):
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(ngram_range=(n, n), binary=True)),
        ('tfidf', TfidfTransformer()),
        ('classifier', RandomForestClassifier())
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    # print(f'Classification Score for n={n} ->{classification_report(y_test, y_pred)}')
    print(f'Accuracy Score for n={n} -> {accuracy_score(y_test, y_pred)}')

    return pipeline

def main():

    def cleaning(df):
        lowered=df.lower() # lowering the sentences 
        removed = re.sub(r'[^a-z]',' ',lowered)  # replacing the non alphabets with space 
        splitted=removed.split(' ')   # splitting the sentences by spaces to lemmatize
        df = [WordNetLemmatizer().lemmatize(word) for word in splitted 
            if word not in stopwords.words('english')]  # lemmatizing and removing stopwords
        df =' '.join(df) # joining back the words of list
        return(removed)

    csv_file = 'data/mohler_dataset_edited.csv'
    data = pd.read_csv(csv_file)
    data['desired_answer'] = data['desired_answer'].apply(cleaning)
    data['student_answer'] = data['student_answer'].apply(cleaning)

    
    text_data = data['student_answer'].tolist()
    labels = data['desired_answer'].tolist() 
    
    # n = 3
    num_n = 6
    for n in range(num_n):
        generate_ngram_tfidf_pipeline(text_data, labels, n)
        
if __name__ == "__main__":
    main()

Accuracy Score for n=0 -> 0.024175824175824177
Accuracy Score for n=1 -> 0.6879120879120879
Accuracy Score for n=2 -> 0.589010989010989
Accuracy Score for n=3 -> 0.46813186813186813
Accuracy Score for n=4 -> 0.33186813186813185
Accuracy Score for n=5 -> 0.23516483516483516
Accuracy Score for n=6 -> 0.16923076923076924
Accuracy Score for n=7 -> 0.12307692307692308
Accuracy Score for n=8 -> 0.1054945054945055
Accuracy Score for n=9 -> 0.0945054945054945


In [11]:
def generate_similarity_scores(data, labels, n):
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(binary=True)),
        ('tfidf', TfidfTransformer()),
    ])

    param_grid = {
        'vectorizer__ngram_range': [(1, n), (2, n)], 
        'vectorizer__min_df': [1, 2, 3],
        'vectorizer__max_df': [0.5, 0.75, 1.0],
        'tfidf__use_idf': [True, False],
    }

    grid_search = GridSearchCV(pipeline, param_grid, cv=5)
    grid_search.fit(data, labels)

    best_params = grid_search.best_params_
    print(f"Best Parameters for n={n}:", best_params)

    pipeline.set_params(**best_params)

    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    pipeline.fit(X_train, y_train)

    # Compute similarity scores
    similarity_scores = cosine_similarity(pipeline.transform(X_test), pipeline.transform(y_test))
    avg_similarity_score = np.mean(similarity_scores.diagonal())

    print(f'Average Cosine Similarity Score for n={n} -> {avg_similarity_score}')

    return pipeline

def main():
    csv_file = 'data/mohler_dataset_edited.csv'
    data = pd.read_csv(csv_file)
    data['desired_answer'] = data['desired_answer'].apply(cleaning)
    data['student_answer'] = data['student_answer'].apply(cleaning)

    text_data = data['student_answer'].tolist()
    labels = data['desired_answer'].tolist()

    num_n = 6  # You can choose your desired n-gram range
    for n in range(1, num_n + 1):
        generate_similarity_scores(text_data, labels, n)

if __name__ == "__main__":
    main()


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('vectorizer', CountVectorizer(binary=True)),
                ('tfidf', TfidfTransformer())]) does not.

In [14]:


def cleaning(text):
    # Lowercasing the text
    lowered = text.lower() 
    
    # Removing non-alphabetic characters
    removed = re.sub(r'[^a-z]', ' ', lowered)  
    
    # Tokenizing the text
    tokens = nltk.word_tokenize(removed)
    
    # Lemmatizing and removing stopwords
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    
    # Joining the tokens back into text
    cleaned_text = ' '.join(cleaned_tokens)
    
    return cleaned_text

# csv_file = 'data/mohler_dataset_edited.csv'
# data = pd.read_csv(csv_file)

# # Applying cleaning function to 'desired_answer' and 'student_answer' columns
# data['desired_answer'] = data['desired_answer'].apply(cleaning)
# data['student_answer'] = data['student_answer'].apply(cleaning)

# # Printing the cleaned data
# print(data)


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def cosine_similarity_score(estimator, X, y):
    # Compute cosine similarity between transformed test data and labels
    similarity_scores = cosine_similarity(estimator.transform(X), estimator.transform(y))
    avg_similarity_score = np.mean(similarity_scores.diagonal())
    return avg_similarity_score

def generate_similarity_scores(data, labels, n):
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(binary=True)),
        ('tfidf', TfidfTransformer()),
    ])

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'vectorizer__ngram_range': [(1, n), (2, n)],  # Vary the ngram range
        'vectorizer__min_df': [1, 2, 3],
        'vectorizer__max_df': [0.5, 0.75, 1.0],
        'tfidf__use_idf': [True, False],
    }

    # Perform grid search with custom scoring function
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=cosine_similarity_score)
    grid_search.fit(data, labels)

    # Get best parameters
    best_params = grid_search.best_params_
    print(f"Best Parameters for n={n}:", best_params)

    # Use the best parameters to build the pipeline
    pipeline.set_params(**best_params)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Fit pipeline on training data
    pipeline.fit(X_train, y_train)

    # Compute similarity scores
    similarity_scores = cosine_similarity(pipeline.transform(X_test), pipeline.transform(y_test))
    avg_similarity_score = np.mean(similarity_scores.diagonal())

    print(f'Average Cosine Similarity Score for n={n} -> {avg_similarity_score}')

    return pipeline

def main():
    csv_file = 'data/mohler_dataset_edited.csv'
    data = pd.read_csv(csv_file)
    data['desired_answer'] = data['desired_answer'].apply(cleaning)
    data['student_answer'] = data['student_answer'].apply(cleaning)

    text_data = data['student_answer'].tolist()
    labels = data['desired_answer'].tolist()

    num_n = 6  # You can choose your desired n-gram range
    for n in range(1, num_n + 1):
        generate_similarity_scores(text_data, labels, n)

if __name__ == "__main__":
    main()


/Users/prabinapokharel/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/prabinapokharel/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/prabinapokharel/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/prabinapokharel/anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 416, in fit
  

Best Parameters for n=1: {'tfidf__use_idf': False, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 1)}
Average Cosine Similarity Score for n=1 -> 0.3545914727519746
Best Parameters for n=2: {'tfidf__use_idf': False, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 2)}
Average Cosine Similarity Score for n=2 -> 0.30732864760267486
Best Parameters for n=3: {'tfidf__use_idf': False, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 3)}
Average Cosine Similarity Score for n=3 -> 0.2983127088398887
Best Parameters for n=4: {'tfidf__use_idf': False, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 4)}
Average Cosine Similarity Score for n=4 -> 0.295664266106952
Best Parameters for n=5: {'tfidf__use_idf': False, 'vectorizer__max_df': 0.5, 'vectorizer__min_df': 3, 'vectorizer__ngram_range': (1, 5)}
Average Cosine Similarity Score for n=5 -> 0.29448990518178186